In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Part 1

In [3]:
import numpy as np
from copy import deepcopy

dirs=[ (0,1), (-1,0), (0,-1), (1,0)]

class PathState:
    def __init__(self,loc,steps ,came_from=None):
        self.loc = loc
        self.steps = steps
        self.came_from = came_from

    def give_neighbors(self,dims, corrupt_locs):
        neighs=[]
        nX,nY = dims
        x,y = self.loc

        for change in dirs:
            nx,ny = change
            ix = x+nx
            iy = y+ny
            if ix <0 or ix > nX or iy < 0 or iy > nY:
                continue
            if (ix,iy) in corrupt_locs:
                continue
                
            neighs.append(  PathState( (ix,iy),
                                self.steps+1, 
                                self)
                         )

        return neighs
                   
    def __repr__(self):
        return f'({self.loc[0]},{self.loc[1]}) with {self.steps}'

    def __str__(self):
        locs=[str(i) for i in self.loc]
        return f'{"-".join(locs)}_{self.steps}'
    
    def __lt__(self,other):
        return self.steps < other.steps

In [4]:
import heapq

class PriorityQueue:
    def __init__(self):
        self.elements: list[tuple[float, PathState]] = []
    
    def empty(self) -> bool:
        return not self.elements
    
    def put(self, item: PathState, priority: float):
        heapq.heappush(self.elements, (priority, item))
    
    def get(self) -> PathState:
        return heapq.heappop(self.elements)[1]

In [11]:
from collections import deque

def perform_breadth_first(dims,corrupted):
     # print out what we find
    frontier = PriorityQueue()
    start = (0,0)
    s= PathState( start, 0 )
    
    frontier.put( s , priority=0)
    
    seen= {start:0}
    
    while not frontier.empty() :
        cur = frontier.get()
        
        if cur.loc == dims:
            return cur.steps
        
        for n_loc in cur.give_neighbors(dims, corrupted):
            
            if n_loc.loc not in seen.keys() or seen[n_loc.loc] > n_loc.steps:
                seen[n_loc.loc]=n_loc.steps
                frontier.put(n_loc, n_loc.steps)

    return -1

In [13]:
def initiate_read(path):
    dims = (6,6) if 'test' in path else (70,70)
    ncorrupt = 12 if 'test' in path else 1024
    
    with open(path,'r') as f:
        corrupted=[]
        for line in f.readlines():
            x,y =line.strip().split(',')
            corrupted.append( (int(x),int(y)) )
            
    return dims, ncorrupt,corrupted


In [14]:
def part_one(path='input_data/test_18.txt'):
    dims, ncorrupt ,corrupted = initiate_read(path)
    
    steps = perform_breadth_first(dims, set(corrupted[:ncorrupt]) )
    return steps

In [15]:
%%time
part_one()

CPU times: user 855 µs, sys: 1.27 ms, total: 2.13 ms
Wall time: 1.29 ms


22

In [16]:
%%time
part_one('input_data/day_18.txt')

CPU times: user 28.7 ms, sys: 3.03 ms, total: 31.8 ms
Wall time: 30.5 ms


310

# Part 2

In [26]:
def binary_search_min(dims ,corrupted):
    left, right = 0, len(corrupted) - 1

    while left <=right:
        ncor = (left+right)//2
        
        steps = perform_breadth_first(dims, set(corrupted[:ncor]) )

        if steps == -1:
            right = ncor
        else:
            left = ncor
            
        if right-left == 1:
            return left
    return -1 
            


In [27]:
def part_two(path='input_data/test_18.txt'):
    dims, ncorrupt ,corrupted = initiate_read(path)

    block = binary_search_min(dims ,corrupted)

    return corrupted[block]

In [28]:
%%time
part_two()

CPU times: user 1.21 ms, sys: 1.42 ms, total: 2.63 ms
Wall time: 2.09 ms


(6, 1)

In [29]:
%%time
part_two('input_data/day_18.txt')

CPU times: user 41.2 ms, sys: 4.52 ms, total: 45.8 ms
Wall time: 50.5 ms


(16, 46)